In [1]:
import numpy as np
from scipy import integrate
from scipy.optimize import root
import scipy.special as sp
import scipy.stats as stats
# from google.colab import drive
import math
import random as rnd
import matplotlib.pyplot as plt
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Two global variables
from typing import Final

posinf: Final[float] = float('inf')
neginf: Final[float] = -float('inf')

In [3]:
# General random variable

class RandomVariable:

  def __init__(self, exp, var):

    if var <= 0:

      raise ValueError('Invalid object parameter! Variance should be a positive real number!')

    else:

      self.expectation = exp
      self.variance = var

  def exp(self):

    return self.expectation

  def var(self):

    return self.variance

  def add(self, other): # A function to add up random variables

    self = RandomVariable(self.exp(), self.var())

    other = RandomVariable(other.exp(), other.var())

    return RandomVariable(self.exp() + other.exp(), self.var() + other.var())

  def __add__(self, other):

    return self.add(other)

  def sub(self, other):

    self = RandomVariable(self.exp(), self.var())

    other = RandomVariable(other.exp(), other.var())

    return RandomVariable(self.exp() - other.exp(), self.var() + other.var())

  def __sub__(self, other):

    return self.sub(other)

  def mul(self, c):

    if isinstance(c, (float, int)) and c != 0:

      return RandomVariable(self.exp() * c, self.var() * c**2)

    else:

      raise TypeError('Unsupported type for this operation! You should provide a non-zero real number!')

  def __mul__(self, c):

    return self.mul(c)

  def div(self, c):

    if isinstance(c, (float, int)) and c != 0:

      return RandomVariable(self.exp()/c, self.var() * 1/c**2)

    else:

      raise TypeError('Unsupported type for this operation! You should provide a non-zero real number!')

  def __truediv__(self, c):

    return self.div(c)

In [4]:
# General continuous distribution

class continuous(RandomVariable):

  def __init__(self, mass_function, lower = neginf, upper = posinf):

    self.mass_function = mass_function
    self.lower = lower
    self.upper = upper

  def var(self):

    return self.nth_moment(2) - self.exp()**2

  def exp(self):

    return self.nth_moment(1)

  def nth_moment(self, n):

    g = lambda x: x**n*self.mass_function(x)

    res, error = integrate.quad(g, self.lower, self.upper)

    return res

  def prob(self, a, b):

    if a < self.lower or b > self.upper:

      raise ValueError('Invalid function parameters! You should provide values between the lower and the upper limit!')

    res, error = integrate.quad(self.mass_function, a, b)

    return res

In [5]:
# Classes of notable continuous distributions with sample functions (that use built-in functions to make samples)

In [6]:
# Exponential distribution

class Exp(continuous):

  def __init__(self, lam, mass_function = 0, lower = 0, upper = posinf):

    if lam <= 0:

      raise ValueError('Invalid object parameter! You should provide a positive real number!')

    else:
      self.lam = lam
      self.mass_function = lambda x: self.lam*math.e**(-self.lam*x)
      self.lower = lower
      self.upper = upper

  def var(self):
    return 1/(self.lam**2)

  def exp(self):
    return 1/self.lam

  def nth_moment(self, n):

    return math.gamma(n+1)/self.lam**n

  def sample(self, n):

    f = lambda x: -math.log(1-x)/self.lam

    return np.array([f(rnd.random()) for _ in range(n)])

  def __add__(self, other):

    if isinstance(other, Exp) and self.lam == other.lam:

      return Gamma(2, self.lam)

    else:

      return super().__add__(other)

  def __mul__(self, c):

    if isinstance(c, (float, int)) and c > 0:

      return Gamma(c, self.lam)

    elif isinstance(c, (float, int)) and c < 0:

      return super().__mul__(c)

    else:

      raise TypeError('Unsupported type for this function! You should provide a non-zero real number!')

  def __truediv__(self, c):

    if isinstance(c, (float, int)) and c > 0:

      return Gamma(1/c, self.lam)

    elif isinstance(c, (float, int)) and c < 0:

      return super().__truediv__(c)

    else:

      raise TypeError('Unsupported type for this function! You should provide a non-zero real number!')

In [7]:
# Gamma distribution

class Gamma(continuous):

  def __init__(self, alpha, lam, mass_function =  0, lower = 0, upper = posinf):

    if alpha <= 0 or lam <= 0:

      raise ValueError('Invalid object parameter(s)! You should provide positive real numbers!')

    else:

      self.alpha = alpha
      self.lam = lam
      self.mass_function = lambda x: self.lam**self.alpha/math.gamma(self.alpha)*x**(self.alpha-1)*math.e**(-self.lam*x)
      self.lower = lower
      self.upper = upper

  def var(self):

    return self.alpha/(self.lam**2)

  def exp(self):

    return self.alpha/self.lam

  def nth_moment(self, n):

    return math.gamma(self.alpha+n)/(self.lam**n*math.gamma(self.alpha))

  def sample(self, n):

    return np.array([stats.gamma.rvs(self.alpha, scale = 1/self.lam, size = n)])

In [8]:
# Uniform distribution

class Uni(continuous):

  def __init__(self, lower, upper, mass_function = 0):

    if lower == upper:

      raise ValueError('Invalid object parameters! Parameters must differ!')

    elif lower > upper:

      lower, upper = upper, lower

    else:

      self.lower = lower
      self.upper = upper
      self.mass_function = 1/(upper-lower)

  def var(self):

    return (self.upper-self.lower)**2/12

  def exp(self):

    return (self.lower+self.upper)/2

  def nth_moment(self, n):

    s = 0

    for i in range(n+1):
      s += self.lower**i*self.upper**(n-i)

    return s/(n+1)

  def sample(self, n):

    return np.array([stats.uniform.rvs(loc = self.a, scale = self.b-self.a, size = n)])

In [9]:
# Normal distribution

class Normal(continuous):

  def __init__(self, m, sigmasq):

    if sigmasq <= 0:

      raise ValueError('Invalid parameter! Variance must be positive!')

    else:

      self.m = m
      self.sigmasq = sigmasq

  def var(self):

    return self.sigmasq

  def exp(self):

    return self.m

  def nth_moment(self, n):

    if n == 1:

      return self.m

    elif self.m == 0 and n%2 == 1:

      return 0

    elif n == 2:

      return self.sigmasq + self.m**2

    elif n == 3:

      return self.m**3 + 3*self.m*self.sigmasq

    elif n == 4:

      return self.m**4 + 6*self.m**2*self.sigmasq + 3*self.sigmasq**2

    else:

      dens = lambda x: (x**n)*1/((math.sqrt(self.sigmasq))*math.sqrt(2*math.pi))*math.e**(-(x-self.m)**2/(2*self.sigmasq))

      res, error = integrate.quad(dens, neginf, posinf)

      return res

  def sample(self, n):

    return np.array([stats.norm.rvs(loc = self.m, scale = math.sqrt(self.sigmasq), size = n)])

  def __add__(self, other):

    if isinstance(other, Normal):

      return Normal(self.exp() + other.exp(), self.var() + other.var())

    else:

      return super().__add__(other)

In [10]:
# Beta distribution

class Beta(continuous):

  def __init__(self, alpha, beta, mass_function = 0, lower = 0, upper = 1):

    if alpha <= 0 or beta <= 0:

      raise ValueError('Invalid object parameters! You should provide positive real numbers!')

    self.alpha = alpha
    self.beta = beta
    self.mass_function = lambda x: math.gamma(alpha+beta)/(math.gamma(alpha)*math.gamma(beta))*x**(alpha-1)*(1-x)**(beta-1)
    self.lower = lower
    self.upper = upper

  def var(self):

    return self.alpha*self.beta/((self.alpha+self.beta)**2*(self.alpha+self.beta+1))

  def exp(self):

    return self.alpha/(self.alpha+self.beta)

  def nth_moment(self, n):

    g = lambda x: x**n*self.mass_function(x)

    res, error = integrate.quad(g, self.lower, self.upper)

    return res

  def sample(self, n):

    return np.array([stats.beta.rvs(self.alpha, self.beta, size = n)])

In [11]:
# General discrete distribution

class discrete(RandomVariable):

  def __init__(self, values, probabilities):

    if not all(0 <= x <= 1 for x in probabilities):

      raise ValueError('Invalid distribution! Each value must be between 0 and 1!')

    else:

      self.values = values
      self.probabilities = probabilities

  def exp(self):

    return self.nth_moment(1)

  def nth_moment(self, n):

    return np.dot(np.array(self.values)**n, np.array(self.probabilities))

  def var(self):

    return self.nth_moment(2) - self.exp()**2

In [12]:
# Classes of some notable discrete distributions with sample functions (that use built-in functions to make samples)

In [13]:
# Geometric distribution

class Geo(discrete):

  def __init__(self, p, values = 0, probabilities = 0):

    if p < 0 or p > 1:

      raise ValueError('Invalid object parameter! You should provide a real number between 0 and 1!')

    else:

      self.p = p

  def exp(self):

    return 1/self.p

  def var(self):

    return (1-self.p)/self.p**2

  def prob(self, val):

    if val > 0 and isinstance(val, int):

      return (1-self.p)**(val-1)*self.p

    else:

      raise ValueError('Invalid function parameter! Geometric distributed random variables can only have positive integer values.')

  def sample(self, n):

    return np.array([stats.geom.rvs(self.p, size = n)])

  def __add__(self, other):

    if isinstance(other, Geo) and self.p == other.p:

      return Negbin(2, self.p)

    else:

      return super().__add__(other)

In [14]:
# Poisson distribution

class Poisson(discrete):

  def __init__(self, lam, values = 0, probabilities = 0):

    if lam <= 0:

      raise ValueError('Invalid object parameter! Lambda should be a positive real number!')

    else:

      self.lam = lam

  def exp(self):

    return self.lam

  def var(self):

    return self.lam

  def prob(self, val):

    if val >= 0 and isinstance(val, int):

      return self.lam**val/math.gamma(val+1)*math.e**(-self.lam)

    else:

      raise ValueError('Invalid function parameter! Poisson distributed random variables can only have nonnegative integer values!')

  def sample(self, n):

    return np.array([stats.poisson.rvs(self.lam, size = n)])

  def __add__(self, other):

    if isinstance(other, Poisson):

      return Poisson(self.lam + other.lam)

    else:

      return super().__add__(other)

In [15]:
# Binomial distribution

class Binom(discrete):

  def __init__(self, n, p, values = 0, probabilities = 0):

    if n <= 0 or p < 0 or p > 1:

      raise ValueError('Invalid object parameter(s)! You should provide a positive integer and a real number between 0 and 1!')

    else:

      self.n = n
      self.p = p

  def exp(self):

    return self.n*self.p

  def var(self):

    return self.n*self.p*(1-self.p)

  def prob(self, val):

    if val <= self.n and val >= 0 and isinstance(val, int):

      return math.comb(self.n, val)*self.p**val*(1-self.p)**(self.n - val)

    else:

      raise ValueError('Invalid function parameter! You should provide a positive integer between 0 and n (the number that you gave earlier)!')

  def sample(self, N):

    return np.array([stats.binom.rvs(self.n, self.p, size = N)])

  def __add__(self, other):

    if isinstance(other, Binom) and self.p == other.p:

      return Binom(self.n + other.n, p)

    else:

      return super().__add__(other)

In [16]:
# Bernoulli distribution

class Ind(discrete):

  def __init__(self, p, values = 0, probabilities = 0):

    if p < 0 or p > 1:

      raise ValueError('Invalid object parameter! You should provide a real number between 0 and 1!')

    else:

      self.p = p

  def exp(self):

    return self.p

  def var(self):

    return self.p*(1-self.p)

  def prob(self, val):

    if val == 0:

      return 1-self.p

    elif val == 1:

      return self.p

    else:

      raise ValueError('Invalid function parameter! A Bernoulli distribution variable can be only 0 or 1!')

  def sample(self, n):

    return np.array([stats.bernoulli.rvs(self.p, size = n)])

  def __add__(self, other):

    if isinstance(other, Ind) and self.p == other.p:

      return Binom(2, self.p)

    else:

      return super().__add__(other)

In [17]:
# Hypergeometric distribution

class Hypergeo(discrete):

  def __init__(self, N, K, n, values = 0, probabilities = 0):

    if N <= 0 or K <= 0 or n <= 0 or K > N or n > N or not isinstance(N, int) or not isinstance(K, int) or not isinstance(n, int):

      raise ValueError('Invalid object parameter(s)! You should provide three positive integers!')

    else:

      self.N = N
      self.K = K
      self.n = n

  def exp(self):

    return self.n*self.K/self.N

  def var(self):

    return self.n*self.K*(self.N - self.K)*(self.N - self.n)/(self.N**2*(self.N - 1))

  def prob(self, val):

    if val <= self.n and isinstance(val, int):

      return math.comb(self.K, val)*math.comb(self.N - self.K, self.n - val)/math.comb(self.N, self.n)

    else:

      raise ValueError('Invalid function parameter! You should provide an integer between 0 and n (the value that you gave earlier)!')

  def sample(self, s):

    return np.array([stats.hypergeom.rvs(self.N, self.K, self.n, size = s)])

In [18]:
# Negative binomial distribution

class Negbin(discrete):

  def __init__(self, r, p, values = 0, probabilities = 0):

    if r <= 0 or not isinstance(r, int) or p < 0 or p > 1:

      raise ValueError('Invalid object parameter(s)! You should provide a positive integer and a real number between 0 and 1!')

    else:

      self.r = r
      self.p = p

  def exp(self):

    return self.r/self.p

  def var(self):

    return self.r*(1-self.p)/self.p**2

  def prob(self, val):

    if val >= self.r and isinstance(val, int):

      return math.comb(val-1, self.r - 1)*self.p**self.r*(1 - self.p)**(val-self.r)

    else:

      raise ValueError('Invalid fuction parameter! You should provide an integer which is greater than r (the value that you gave earlier)!')

  def sample(self, n):

    return np.array([stats.nbinom.rvs(self.r, self.p, size = n)])

  def __add__(self, other):

    if isinstance(other, Negbin) and self.p == other.p:

      return Negbin(self.r + other.r, self.p)

    else:

      return super().__add__(other)

In [19]:
# Simple functions to work with samples:

In [43]:
# Corrected Sample Standard Deviation

def Cssd(sample):

  s = np.array(sample)

  if len(s) != 1:

    return math.sqrt(((s - s.mean())**2).sum()/(len(s)-1))

  else:

    return math.sqrt(((s - s.mean())**2).sum()/(s.shape[1]-1))

In [44]:
# Empirical covariance of two samples

def Sample_cov(samp1, samp2):

  samp1 = np.array(samp1)

  samp2 = np.array(samp2)

  samp1 = samp1 - samp1.mean()

  samp2 = samp2 - samp2.mean()

  samp2 = np.transpose(samp2)

  return np.dot(samp1, samp2)/(samp1.shape[1] - 1)

In [45]:
# Empirical covariance of two specific random variables from the built-in ones

def Cov(X, Y):

    return Sample_cov(X.sample(1000), Y.sample(1000))

In [46]:
# Empirical correlation of two samples

def Sample_cor(samp1, samp2):

  return Sample_cov(samp1, samp2)/(Cssd(samp1)*Cssd(samp2))

In [47]:
# Empirical correlation of two random variables

def Cor(X,Y):

  return Cov(X,Y)/(math.sqrt(X.var())*math.sqrt(Y.var()))

In [48]:
# Statistical tests:

ac = "Null hypothesis must be accepted!"

rej = "Null hypothesis must be rejected!"

In [49]:
# one sample u test:

def one_sample_u_test(sample, sigma, mu0, alpha, alternative):

  samp = np.array(sample)

  X = samp.mean()

  T = math.sqrt(len(samp))*(X - mu0)/sigma

  if alternative == 'greater':

    if T < -stats.norm.ppf(1-alpha):

      return rej

    else:

      return ac

  elif alternative == 'less':

    if T > stats.norm.ppf(1-alpha):

      return rej

    else:

      return ac

  elif alternative == 'not_equal':

    if abs(T) > stats.norm.ppf(1-alpha/2):

      return rej

    else:

      return ac

  else:

    raise ValueError('Alternative can be greater, less or not_equal!')

In [50]:
# two sample u test:

def two_sample_u_test(samp1, samp2, sigma1, sigma2, mu1, mu2, alpha, alternative):

  samp1 = np.array(samp1)

  samp2 = np.array(samp2)

  X = samp1.mean()

  Y = samp2.mean()

  T = (X - Y)/math.sqrt(sigma1**2/len(samp1) + sigma2**2/len(samp2))

  if alternative == 'greater':

    if T < -stats.norm.ppf(1-alpha):

      return rej

    else:

      return ac

  elif alternative == 'less':

    if T > stats.norm.ppf(1-alpha):

      return rej

    else:

      return ac

  elif alternative == 'not_equal':

    if abs(T) > stats.norm.ppf(1-alpha/2):

      return rej

    else:

      return ac

  else:

    raise ValueError('Alternative can be greater, less or not_equal!')

In [51]:
# One sample t test:

def one_sample_t_test(sample, mu0, alpha, alternative):

  sample = np.array(sample)

  X = sample.mean()

  T = math.sqrt(len(sample))*(X - mu0)/Cssd(sample)

  df = len(sample) - 1 # degrees of freedom

  if alternative == 'greater':

    if T < -stats.t.ppf(1-alpha, df):

      return rej

    else:

      return ac

  elif alternative == 'less':

    if T > stats.t.ppf(1-alpha, df):

      return rej

    else:

      return ac

  elif alternative == 'not_equal':

    if abs(T) > stats.t.ppf(1-alpha/2, df):

      return rej

    else:

      return ac

  else:

    raise ValueError('Alternative can be greater, less or not_equal!')

In [52]:
# two sample t test

def two_sample_t_test(samp1, samp2, alpha, alternative):

  samp1 = np.array(samp1)

  samp2 = np.array(samp2)

  sigma = (((samp1 - samp1.mean())**2).sum() + ((samp2 - samp2.mean())**2).sum())/(len(samp1) + len(samp2) - 2)

  X = samp1.mean()

  Y = samp2.mean()

  T = (X - Y)/math.sqrt(sigma**2/len(samp1) + sigma**2/len(samp2))

  df = len(samp1) + len(samp2) - 2

  if alternative == 'greater':

    if T < -stats.t.ppf(1-alpha, df):

      return rej

    else:

      return ac

  elif alternative == 'less':

    if T > stats.t.ppf(1-alpha, df):

      return rej

    else:

      ac

  elif alternative == "not_equal":

    if abs(T) > stats.t.ppf(1-alpha/2, df):

      return rej

    else:

      return ac

  else:

    raise ValueError('Alternative can be greater, less or not_equal!')

In [53]:
# F test:

def F_test(samp1, samp2, alpha, alternative):

  sigma1 = Cssd(samp1)**2

  sigma2 = Cssd(samp2)**2

  T = sigma1/sigma2

  df1 = len(samp1) - 1 # Degrees of freedom

  df2 = len(samp2) - 1

  if alternative == 'greater':

    if T > stats.f.ppf(1-alpha, df1, df2):

      return rej

    else:

      return ac

  elif alternative == 'not_equal':

    if T > stats.f.ppf(1-alpha/2, df1, df2) or T < stats.f.ppf(alpha/2, df1, df2):

      return rej

    else:

      return ac

  else:

    raise ValueError('Alternative can be greater, less or not_equal!')